# Preprocessing files

### Importing libraries

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

### Creating list of csv files

In [2]:
print(os.getcwd())

filepath = os.getcwd()+'/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

C:\Users\shubham


In [3]:
file_path_list

['C:\\Users\\shubham/event_data\\2018-11-01-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-02-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-03-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-04-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-05-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-06-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-07-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-08-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-09-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-10-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-11-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-12-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-13-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-14-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-15-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-16-events.csv',
 'C:\\Users\\shubham/event_data\\2018-11-17-events.csv',
 'C:\\Users\\shubham/event_data

### Processing individual csv's to create csv that we will use for apache cassandra

In [4]:
full_data_rows = []

for f in file_path_list:
    with open(f, 'r', encoding='utf-8', newline='') as csvFile:
        csvReader = csv.reader(csvFile)
        next(csvReader)
        
        for line in csvReader:
            full_data_rows.append(line)
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('events_combined.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    
    for row in full_data_rows:
        if row[0] == '':
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
with open('events_combined.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Creating key space and cluster

In [6]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

In [8]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)

In [9]:
session.set_keyspace('sparkify')

# Query #1

Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

To answer this question we will need to obtain (select) the artist name, song name, and song length from out table, and we will need to filter by sessionId and itemInSession. In CQL our query looks like:

```SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4```

We will name our table session_songs

Our primary key will consist of partition key sessionId, and clustering key itemInSession so that we can filter by this attributes later on.

The columns of our table will be: sessionId, itemInSession, artist, song_title and song_length.

In [11]:
session.execute("""
    CREATE TABLE IF NOT EXISTS session_songs
    (sessionId int, itemInSession int, artist text, song_title text, song_length float,
    PRIMARY KEY(sessionId, itemInSession))
    """)

In [12]:
file = './events_combined.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_songs (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist_name, song, float(length)))

In [13]:
rows = session.execute("""SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4""")

for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Query #2

Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To answer this question we will need to obtain (select) the artist name, song name, user name and user lastname from out table, we will need to filter by userId and sessionId, and order by itemInSession.

In CQL our query looks like:

```SELECT itemInSession, artist, song, firstName, lastName FROM user_songs WHERE userId = 10 AND sessionId = 182```

We will name our table user_songs

Our primary key will consist of composite partition key userId, sessionId. The reason for this is that if we only use userId as partition key, the sessionid which belongs to the same user will be put into different nodes, which will have the performance issue when the volume of data is large.

Our clustering key will be itemInSession so that our results are order by it.

The columns of our table will be: userId, sessionId, itemInSession, artist, song and firstName and lastName.


In [14]:
session.execute("""
    CREATE TABLE IF NOT EXISTS user_songs
    (userId int, sessionId int, artist text, song text, firstName text, lastName text, itemInSession int,
    PRIMARY KEY((userId, sessionId), itemInSession))
    """)

In [16]:
file = './events_combined.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs (userId, sessionId, artist, song, firstName, lastName, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(userId), int(sessionId), artist, song, firstName, lastName, int(itemInSession)))

In [17]:
rows = session.execute("""SELECT itemInSession, artist, song, firstName, lastName FROM user_songs WHERE userId = 10 AND sessionId = 182""")

for row in rows:
    print(row.iteminsession, row.artist, row.song, row.firstname, row.lastname)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


# Query #3

Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

To answer this question we will need to obtain (select) the user first name and lastname from out table, and we will need to filter by song name. As user name and lastname, in large datasets, are not unique, we will add the column userId to uniquely identify users.

In CQL our query looks like:

```SELECT firstName, lastName FROM app_history WHERE song = 'All Hands Against His Own'```

We will name our table app_history

Our primary key will consist of partition key song, and clustering key userId. This uniquely identifies our rows.

The columns of our table will be: song, firstName, lastName and userId.

In [18]:
session.execute("""
    CREATE TABLE IF NOT EXISTS app_history
    (song text, firstName text, lastName text, userId int,
    PRIMARY KEY(song, userId))
    """)

In [21]:
file = './events_combined.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO app_history (song, firstName, lastName, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (song, firstName, lastName, int(userId)))

In [22]:
rows = session.execute("""SELECT firstName, lastName FROM app_history WHERE song = 'All Hands Against His Own'""")

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


# Dropping created tables

In [23]:
session.execute("""DROP TABLE app_history""")

session.execute("""DROP TABLE user_songs""")

session.execute("""DROP TABLE session_songs""")

# Close session and cluster

In [24]:
session.shutdown()
cluster.shutdown()